# Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [2]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "/opt/notebooks/data/state-farm-distracted-driver-detection/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=32

In [7]:
ls $path/train/c0

img_1230.jpg   img_19246.jpg  img_40930.jpg  img_65981.jpg  img_87846.jpg
img_15565.jpg  img_20994.jpg  img_47634.jpg  img_68180.jpg  img_92203.jpg
img_16494.jpg  img_23996.jpg  img_50577.jpg  img_72977.jpg  img_9678.jpg
img_17811.jpg  img_27618.jpg  img_61788.jpg  img_85072.jpg  img_99012.jpg


## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

%cd data/state

%cd train

%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

from shutil import copyfile

g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

%cd ../valid

g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

%cd ../../..

%mkdir data/state/results

%mkdir data/state/sample/test

## Create batches

In [25]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1492 images belonging to 10 classes.
Found 979 images belonging to 10 classes.


In [11]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 1492 images belonging to 10 classes.
Found 979 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [12]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [13]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1492/1492 [==============================] - 22s - loss: 14.0596 - acc: 0.1099 - val_loss: 14.3191 - val_acc: 0.1113
Epoch 2/2
1492/1492 [==============================] - 19s - loss: 14.3248 - acc: 0.1113 - val_loss: 14.3071 - val_acc: 0.1124


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [14]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [15]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [41]:
np.round(model.predict_generator(batches, batches.n)[:10],2)

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [44]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1492/1492 [==============================] - 22s - loss: 2.2323 - acc: 0.2460 - val_loss: 2.8198 - val_acc: 0.2114
Epoch 2/2
1492/1492 [==============================] - 19s - loss: 1.5117 - acc: 0.5214 - val_loss: 1.9365 - val_acc: 0.3432


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [45]:
model.optimizer.lr=0.001

In [46]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1492/1492 [==============================] - 22s - loss: 1.1162 - acc: 0.6790 - val_loss: 1.2518 - val_acc: 0.5781
Epoch 2/4
1492/1492 [==============================] - 19s - loss: 0.8933 - acc: 0.7849 - val_loss: 1.0714 - val_acc: 0.6762
Epoch 3/4
1492/1492 [==============================] - 19s - loss: 0.7347 - acc: 0.8365 - val_loss: 0.9415 - val_acc: 0.7201
Epoch 4/4
1492/1492 [==============================] - 19s - loss: 0.6195 - acc: 0.8800 - val_loss: 0.8418 - val_acc: 0.7681


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [47]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 979 images belonging to 10 classes.


In [48]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 0.84,  0.77],
       [ 0.84,  0.77],
       [ 0.84,  0.77],
       [ 0.84,  0.77],
       [ 0.85,  0.76],
       [ 0.81,  0.78],
       [ 0.85,  0.78],
       [ 0.84,  0.77],
       [ 0.83,  0.77],
       [ 0.84,  0.76]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [49]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1492/1492 [==============================] - 22s - loss: 5.6825 - acc: 0.3264 - val_loss: 6.2904 - val_acc: 0.3432
Epoch 2/2
1492/1492 [==============================] - 19s - loss: 2.7018 - acc: 0.6434 - val_loss: 3.1265 - val_acc: 0.5853


In [50]:
model.optimizer.lr=0.001

In [51]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1492/1492 [==============================] - 22s - loss: 2.1228 - acc: 0.7929 - val_loss: 2.3950 - val_acc: 0.7099
Epoch 2/4
1492/1492 [==============================] - 19s - loss: 2.1187 - acc: 0.8412 - val_loss: 2.3507 - val_acc: 0.7354
Epoch 3/4
1492/1492 [==============================] - 19s - loss: 1.9096 - acc: 0.8733 - val_loss: 2.1537 - val_acc: 0.8090
Epoch 4/4
1492/1492 [==============================] - 19s - loss: 1.8508 - acc: 0.8968 - val_loss: 2.0797 - val_acc: 0.8182


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [52]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1492/1492 [==============================] - 22s - loss: 1.8635 - acc: 0.3881 - val_loss: 4.0435 - val_acc: 0.2513
Epoch 2/2
1492/1492 [==============================] - 19s - loss: 0.9082 - acc: 0.7446 - val_loss: 1.9797 - val_acc: 0.3820
Epoch 1/5
1492/1492 [==============================] - 22s - loss: 0.5195 - acc: 0.8847 - val_loss: 0.8880 - val_acc: 0.7079
Epoch 2/5
1492/1492 [==============================] - 19s - loss: 0.3171 - acc: 0.9531 - val_loss: 0.6283 - val_acc: 0.8345
Epoch 3/5
1492/1492 [==============================] - 19s - loss: 0.2020 - acc: 0.9819 - val_loss: 0.5404 - val_acc: 0.8754
Epoch 4/5
1492/1492 [==============================] - 19s - loss: 0.1456 - acc: 0.9987 - val_loss: 0.4700 - val_acc: 0.8856
Epoch 5/5
1492/1492 [==============================] - 19s - loss: 0.1006 - acc: 0.9987 - val_loss: 0.3611 - val_acc: 0.9285


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [60]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [61]:
conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 1.4678 - acc: 0.5885 - val_loss: 1.8693 - val_acc: 0.3013
Epoch 2/2
1492/1492 [==============================] - 20s - loss: 0.3280 - acc: 0.9330 - val_loss: 2.3073 - val_acc: 0.1645
Epoch 1/4
1492/1492 [==============================] - 23s - loss: 0.0910 - acc: 0.9913 - val_loss: 2.6375 - val_acc: 0.1696
Epoch 2/4
1492/1492 [==============================] - 20s - loss: 0.0437 - acc: 0.9987 - val_loss: 2.4210 - val_acc: 0.1910
Epoch 3/4
1492/1492 [==============================] - 20s - loss: 0.0187 - acc: 1.0000 - val_loss: 1.9948 - val_acc: 0.2799
Epoch 4/4
1492/1492 [==============================] - 20s - loss: 0.0124 - acc: 1.0000 - val_loss: 1.3897 - val_acc: 0.4913


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [62]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [63]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 2.1081 - acc: 0.3492 - val_loss: 2.2570 - val_acc: 0.2033
Epoch 2/2
1492/1492 [==============================] - 20s - loss: 1.1344 - acc: 0.6535 - val_loss: 2.2897 - val_acc: 0.2257
Epoch 1/4
1492/1492 [==============================] - 23s - loss: 0.7725 - acc: 0.7681 - val_loss: 2.4583 - val_acc: 0.2676
Epoch 2/4
1492/1492 [==============================] - 20s - loss: 0.5990 - acc: 0.8271 - val_loss: 2.4890 - val_acc: 0.2840
Epoch 3/4
1492/1492 [==============================] - 20s - loss: 0.4537 - acc: 0.8686 - val_loss: 2.2959 - val_acc: 0.3115
Epoch 4/4
1492/1492 [==============================] - 20s - loss: 0.3720 - acc: 0.9068 - val_loss: 1.9331 - val_acc: 0.3269


Height shift: move the image up and down -

In [64]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [65]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 1.8756 - acc: 0.4229 - val_loss: 1.8962 - val_acc: 0.3013
Epoch 2/2
1492/1492 [==============================] - 20s - loss: 0.7905 - acc: 0.7748 - val_loss: 2.0157 - val_acc: 0.2308
Epoch 1/4
1492/1492 [==============================] - 23s - loss: 0.4326 - acc: 0.8901 - val_loss: 2.1152 - val_acc: 0.2114
Epoch 2/4
1492/1492 [==============================] - 21s - loss: 0.3213 - acc: 0.9216 - val_loss: 2.1037 - val_acc: 0.2329
Epoch 3/4
1492/1492 [==============================] - 21s - loss: 0.2307 - acc: 0.9450 - val_loss: 1.8009 - val_acc: 0.3667
Epoch 4/4
1492/1492 [==============================] - 21s - loss: 0.1549 - acc: 0.9705 - val_loss: 1.5052 - val_acc: 0.4576


Random shear angles (max in radians) -

In [66]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [67]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 1.5939 - acc: 0.5235 - val_loss: 1.9363 - val_acc: 0.3126
Epoch 2/2
1492/1492 [==============================] - 20s - loss: 0.4387 - acc: 0.8948 - val_loss: 1.7996 - val_acc: 0.4658
Epoch 1/4
1492/1492 [==============================] - 23s - loss: 0.1663 - acc: 0.9732 - val_loss: 1.8483 - val_acc: 0.4402
Epoch 2/4
1492/1492 [==============================] - 20s - loss: 0.1074 - acc: 0.9826 - val_loss: 1.6529 - val_acc: 0.4852
Epoch 3/4
1492/1492 [==============================] - 20s - loss: 0.0629 - acc: 0.9933 - val_loss: 1.3863 - val_acc: 0.5434
Epoch 4/4
1492/1492 [==============================] - 20s - loss: 0.0392 - acc: 0.9980 - val_loss: 1.1502 - val_acc: 0.6394


Rotation: max in degrees -

In [68]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [69]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 1.9397 - acc: 0.4015 - val_loss: 1.9600 - val_acc: 0.3054
Epoch 2/2
1492/1492 [==============================] - 21s - loss: 0.9311 - acc: 0.7031 - val_loss: 2.4896 - val_acc: 0.1982
Epoch 1/4
1492/1492 [==============================] - 23s - loss: 0.5907 - acc: 0.8291 - val_loss: 2.7454 - val_acc: 0.1890
Epoch 2/4
1492/1492 [==============================] - 21s - loss: 0.4201 - acc: 0.8881 - val_loss: 2.6415 - val_acc: 0.2074
Epoch 3/4
1492/1492 [==============================] - 20s - loss: 0.3005 - acc: 0.9216 - val_loss: 2.3820 - val_acc: 0.2686
Epoch 4/4
1492/1492 [==============================] - 21s - loss: 0.2412 - acc: 0.9457 - val_loss: 1.8409 - val_acc: 0.3984


Channel shift: randomly changing the R,G,B colors - 

In [70]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [71]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 23s - loss: 1.5973 - acc: 0.5295 - val_loss: 1.9671 - val_acc: 0.2411
Epoch 2/2
1492/1492 [==============================] - 21s - loss: 0.3453 - acc: 0.9323 - val_loss: 2.1853 - val_acc: 0.2431
Epoch 1/4
1492/1492 [==============================] - 24s - loss: 0.1221 - acc: 0.9832 - val_loss: 2.0035 - val_acc: 0.3657
Epoch 2/4
1492/1492 [==============================] - 21s - loss: 0.0523 - acc: 0.9966 - val_loss: 1.7710 - val_acc: 0.4402
Epoch 3/4
1492/1492 [==============================] - 21s - loss: 0.0299 - acc: 0.9993 - val_loss: 1.5061 - val_acc: 0.5424
Epoch 4/4
1492/1492 [==============================] - 21s - loss: 0.0163 - acc: 0.9993 - val_loss: 1.1494 - val_acc: 0.6404


And finally, putting it all together!

In [72]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1492 images belonging to 10 classes.


In [73]:
model = conv1(batches)

Epoch 1/2
1492/1492 [==============================] - 24s - loss: 2.4319 - acc: 0.2487 - val_loss: 2.2317 - val_acc: 0.2288
Epoch 2/2
1492/1492 [==============================] - 21s - loss: 1.7372 - acc: 0.4082 - val_loss: 2.4512 - val_acc: 0.1971
Epoch 1/4
1492/1492 [==============================] - 24s - loss: 1.4636 - acc: 0.5127 - val_loss: 2.3592 - val_acc: 0.2176
Epoch 2/4
1492/1492 [==============================] - 21s - loss: 1.3439 - acc: 0.5597 - val_loss: 2.2422 - val_acc: 0.2237
Epoch 3/4
1492/1492 [==============================] - 21s - loss: 1.2459 - acc: 0.5992 - val_loss: 2.0603 - val_acc: 0.2523
Epoch 4/4
1492/1492 [==============================] - 21s - loss: 1.0696 - acc: 0.6682 - val_loss: 1.7732 - val_acc: 0.3105


At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [74]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1492/1492 [==============================] - 24s - loss: 0.9984 - acc: 0.6810 - val_loss: 1.5486 - val_acc: 0.4586
Epoch 2/5
1492/1492 [==============================] - 21s - loss: 0.9485 - acc: 0.7031 - val_loss: 1.2245 - val_acc: 0.5914
Epoch 3/5
1492/1492 [==============================] - 21s - loss: 0.8939 - acc: 0.7058 - val_loss: 1.0396 - val_acc: 0.6793
Epoch 4/5
1492/1492 [==============================] - 21s - loss: 0.8557 - acc: 0.7292 - val_loss: 0.9752 - val_acc: 0.6619
Epoch 5/5
1492/1492 [==============================] - 21s - loss: 0.8003 - acc: 0.7493 - val_loss: 0.8694 - val_acc: 0.7406


Lucky we tried that - we starting to make progress! Let's keep going.

In [75]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1492/1492 [==============================] - 24s - loss: 0.7331 - acc: 0.7768 - val_loss: 0.6920 - val_acc: 0.7773
Epoch 2/25
1492/1492 [==============================] - 21s - loss: 0.7117 - acc: 0.7842 - val_loss: 0.7008 - val_acc: 0.7804
Epoch 3/25
1492/1492 [==============================] - 21s - loss: 0.6778 - acc: 0.7909 - val_loss: 0.6303 - val_acc: 0.8121
Epoch 4/25
1492/1492 [==============================] - 21s - loss: 0.7120 - acc: 0.7768 - val_loss: 0.5875 - val_acc: 0.8253
Epoch 5/25
1492/1492 [==============================] - 21s - loss: 0.6495 - acc: 0.7902 - val_loss: 0.4930 - val_acc: 0.8498
Epoch 6/25
1492/1492 [==============================] - 21s - loss: 0.6405 - acc: 0.8050 - val_loss: 0.5636 - val_acc: 0.8202
Epoch 7/25
1492/1492 [==============================] - 21s - loss: 0.6102 - acc: 0.8257 - val_loss: 0.4297 - val_acc: 0.8836
Epoch 8/25
1492/1492 [==============================] - 21s - loss: 0.5691 - acc: 0.8210 - val_loss: 0.4217 - val_acc:

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.